In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [ ]:
pyspark_version = pyspark.__version__
kafka_jar_package = f'org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}'

In [ ]:
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName('GreenTripsConsumer') \
    .config('spark.jars.packages', kafka_jar_package) \
    .getOrCreate()

In [ ]:
green_stream = spark \
    .readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', 'localhost:9092') \
    .option('subscribe', 'green-trips') \
    .option('startingOffsets', 'earliest') \
    .load()

In [ ]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

In [ ]:
schema = types.StructType() \
    .add('lpep_pickup_datetime', types.StringType()) \
    .add('lpep_dropoff_datetime', types.StringType()) \
    .add('PULocationID', types.IntegerType()) \
    .add('DOLocationID', types.IntegerType()) \
    .add('passenger_count', types.DoubleType()) \
    .add('trip_distance', types.DoubleType()) \
    .add('tip_amount', types.DoubleType())

In [ ]:
green_stream = green_stream \
    .select(F.from_json(F.col('value').cast('STRING'), schema).alias('data')) \
    .select('data.*')

In [ ]:
# query = green_stream.writeStream.foreachBatch(peek).start()

In [ ]:
popular_destinations = green_stream \
    .withColumn('timestamp', F.current_timestamp()) \
    .groupBy(F.window(F.col('timestamp'), '5 minutes'), 'DOLocationID') \
    .count() \
    .orderBy('count', ascending=False)

In [ ]:
query2 = popular_destinations \
    .writeStream \
    .outputMode('complete') \
    .format('console') \
    .option('truncate', 'false') \
    .start()
query2.awaitTermination()